<a href="https://colab.research.google.com/github/aecoaker/FTA-Summary/blob/master/Creating%20Final%20Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarisation

In [4]:
#import the model
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
# load a pre-trained model and tokenizer 'bart-large-cnn'
tokeniser = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [8]:
#import other libraries
import pandas as pd

In [ ]:
#write in function developed previously for splitting into sections (or chunks) that are not too long to be fed into BART
def summary_chunks(text):
    chunks = []
    #split and join back together to remove odd whitespaces
    remaining_text = " ".join(text.split())
    #loop through and split text off into the list of chunks
    while remaining_text.count('.') > 1:
      words = remaining_text.split()
      stop_index = " ".join(words[0:750]).rfind('.')
      chunks.append(remaining_text[0:stop_index+1])
      remaining_text = remaining_text[stop_index+1:]
    #have to remove some specific sections which tokenise into too many words in an unhelpful way in BART's tokeniser
    chunks = [c.replace('𝑅𝑅𝑅𝑅𝑅𝑅 = 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑡𝑡ℎ𝑣𝑣 𝑔𝑔𝑜𝑜𝑜𝑜𝑔𝑔 − 𝑣𝑣𝑣𝑣 𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑛𝑛𝑜𝑜𝑛𝑛−𝑜𝑜𝑜𝑜𝑜𝑜𝑔𝑔𝑜𝑜𝑛𝑛𝑣𝑣𝑡𝑡𝑜𝑜𝑛𝑛𝑔𝑔 𝑚𝑚𝑣𝑣𝑡𝑡𝑣𝑣𝑜𝑜 𝑜𝑜𝑣𝑣𝑣𝑣𝑚𝑚 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑡𝑡ℎ𝑣𝑣 𝑔𝑔𝑜𝑜𝑜𝑜𝑔𝑔', '') for c in chunks]
    chunks = [c.replace('𝑅𝑅𝑅𝑅𝑅𝑅 = 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑜𝑜𝑜𝑜𝑜𝑜𝑔𝑔𝑜𝑜𝑛𝑛𝑣𝑣𝑡𝑡𝑜𝑜𝑛𝑛𝑔𝑔 𝑚𝑚𝑣𝑣𝑡𝑡𝑣𝑣𝑜𝑜 𝑜𝑜𝑣𝑣𝑣𝑣𝑚𝑚 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑡𝑡ℎ𝑣𝑣 𝑔𝑔𝑜𝑜𝑜𝑜𝑔𝑔×100', '') for c in chunks]
    return chunks

In [9]:
articles_df = pd.read_csv("article_topics.csv")

In [14]:
#generate summaries for app
l = 0.9
summaries = []
pre_recursive_summaries = []
for t in range(20):
  example_classification = articles_df[articles_df[('prob_t' + str(t))] >= l]
  example_topic_text = ' '.join(example_classification['Article Text'])
  example_text_chunks = summary_chunks(example_topic_text)
  inputs = [tokeniser.batch_encode_plus([text],return_tensors = 'pt') for text in example_text_chunks]
  summary_ids = [model.generate(i['input_ids'], early_stopping = True) for i in inputs]
  bart_summaries = [tokeniser.decode(ids[0], skip_special_tokens = True) for ids in summary_ids]
  single_sum = ' '.join(bart_summaries)
  pre_recursive_summaries.append(single_sum)
  input = tokeniser.batch_encode_plus([single_sum],return_tensors='pt')
  summary_id = model.generate(input['input_ids'], early_stopping=True)
  final_sum = tokeniser.decode(summary_id[0], skip_special_tokens=True)
  print(str(t)) #print the step in order to keep track of progress
  summaries.append(final_sum)
print(summaries)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0
1
2
3
4


KeyboardInterrupt: ignored

In [19]:
#generate summaries for app
l = 0.9
summaries_1 = []
pre_recursive_summaries = []
for t in range(20):
  example_classification = articles_df[articles_df[('prob_t' + str(t))] >= l]
  example_topic_text = ' '.join(example_classification['Article Text'])
  example_text_chunks = summary_chunks(example_topic_text)
  inputs = [tokeniser.batch_encode_plus([text],return_tensors = 'pt') for text in example_text_chunks]
  summary_ids = [model.generate(i['input_ids'], min_length = 250, early_stopping = True) for i in inputs]
  bart_summaries = [tokeniser.decode(ids[0], skip_special_tokens = True) for ids in summary_ids]
  single_sum = ' '.join(bart_summaries)
  pre_recursive_summaries.append(single_sum)
  input = tokeniser.batch_encode_plus([single_sum],return_tensors='pt')
  summary_id = model.generate(input['input_ids'], early_stopping=True)
  final_sum = tokeniser.decode(summary_id[0], skip_special_tokens=True)
  print(str(t)) #print the step in order to keep track of progress
  summaries.append(final_sum)
print(summaries)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ValueError: ignored

In [16]:
summaries

['Except as provided in this Section, e ach Party affirms its rights and obligations',
 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on',
 'Each Party shall accord national treatment to the goods of the other Party in accordance with Article',
 'A Party shall not reject a declaration of origin due to minor errors or discrepancies, such',
 'Expressions The Parties affirm their commitment to work together through discussion and by the exchange of',
 'Except as provided in this Section, e ach Party affirms its rights and obligations',
 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on',
 'Each Party shall accord national treatment to the goods of the other Party in accordance with Article',
 'A Party shall not reject a declaration of origin due to minor errors or discrepancies, such',
 'Expressions The Parties affirm their commitment to work together through discussion and by the exchange of']

In [18]:
summaries_1

[]